<a href="https://colab.research.google.com/github/BrunoAlexandreAmaral/modelo_clasificacao/blob/main/tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import pandas as pd
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,roc_auc_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
uploaded = files.upload()

Saving Bank_Personal_Loan_Modelling.xlsx to Bank_Personal_Loan_Modelling.xlsx


In [35]:
df = pd.read_excel("Bank_Personal_Loan_Modelling.xlsx")

In [31]:
df.head()

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,1,25,1,49,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,4,1.0,2,0,0,0,0,0,1


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   Family              5000 non-null   int64  
 5   CCAvg               5000 non-null   float64
 6   Education           5000 non-null   int64  
 7   Mortgage            5000 non-null   int64  
 8   Personal_Loan       5000 non-null   int64  
 9   Securities_Account  5000 non-null   int64  
 10  CD_Account          5000 non-null   int64  
 11  Online              5000 non-null   int64  
 12  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 507.9 KB


In [6]:
df.isnull().sum()

,0
ID,0
Age,0
Experience,0
Income,0
Family,0
CCAvg,0
Education,0
Mortgage,0
Personal_Loan,0
Securities_Account,0


In [7]:
df.describe()

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,2.396400,1.937913,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,1.147663,1.747666,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [37]:
x = df.drop("Personal_Loan", axis=1)
y = df["Personal_Loan"]

In [38]:
x

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities_Account,CD_Account,Online,CreditCard
0,1,25,1,49,4,1.6,1,0,1,0,0,0
1,2,45,19,34,3,1.5,1,0,1,0,0,0
2,3,39,15,11,1,1.0,1,0,0,0,0,0
3,4,35,9,100,1,2.7,2,0,0,0,0,0
4,5,35,8,45,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,1,1.9,3,0,0,0,1,0
4996,4997,30,4,15,4,0.4,1,85,0,0,1,0
4997,4998,63,39,24,2,0.3,3,0,0,0,0,0
4998,4999,65,40,49,3,0.5,2,0,0,0,1,0


In [39]:
y

,Personal_Loan
0,0
1,0
2,0
3,0
4,0
...,...
4995,0
4996,0
4997,0
4998,0


In [21]:
y.value_counts()

,count
Personal_Loan,
0,4520
1,480


In [40]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [44]:
tree = DecisionTreeClassifier()

In [45]:
tree.fit(x_train,y_train)

DecisionTreeClassifier()

In [68]:
pred_tree = tree.predict(x_test)

In [69]:
pred_tree = pred_tree

In [71]:
x_test.assign(Valor_Real=y_test).assign(Valor_Predito=pred_tree)

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities_Account,CD_Account,Online,CreditCard,Valor_Real,Valor_Predito
1501,1502,30,4,35,2,0.3,2,0,1,0,0,1,0,0
2586,2587,47,23,149,4,6.1,1,0,0,0,0,1,1,1
2653,2654,30,5,121,2,3.1,1,408,0,0,1,0,0,0
1055,1056,31,6,62,1,1.0,1,0,1,0,1,0,0,0
705,706,62,36,30,3,0.7,2,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,4712,65,40,59,3,2.4,1,0,0,0,0,0,0,0
2313,2314,58,32,54,3,0.3,2,0,0,0,1,1,0,0
3214,3215,61,37,33,3,0.1,3,0,0,0,1,0,0,0
2732,2733,33,9,38,1,2.1,3,133,0,0,0,0,0,0


In [108]:

print("METRICAS DO MODLEO DECISION TREE CLASSIFIER (PADRAO)")
print("*"*20)
print(f"Acurancia Train:{accuracy_score(y_train,tree.predict(x_train))}")
print(f"Acurancia teste:{accuracy_score(y_test,tree.predict(x_test))}")
print("*"*20)
print(f"Acuracia balancead Treino: {balanced_accuracy_score(y_train,tree.predict(x_train))}")
print(f"Acuracia balancead Teste: {balanced_accuracy_score(y_test,tree.predict(x_test))}")
print("*"*20)
print(f"Recall Score :{recall_score(y_train,tree.predict(x_train))}")
print(f"Recall Score :{recall_score(y_test,tree.predict(x_test))}")
print("*"*20)
print(f"Roc_Auc treino:{roc_auc_score(y_train,tree.predict(x_train))}")
print(f"Roc_Auc teste:{roc_auc_score(y_test,tree.predict(x_test))}")
print("*"*20)
print(f"Precisao Treino: {precision_score(y_train,tree.predict(x_train))}")
print(f"Precisao: Teste: {precision_score(y_test,tree.predict(x_test))}")
print("*"*20)

METRICAS DO MODLEO DECISION TREE CLASSIFIER (PADRAO)
********************
Acurancia Train:1.0
Acurancia teste:0.985
********************
Acuracia balancead Treino: 1.0
Acuracia balancead Teste: 0.9537909018355946
********************
Recall Score :1.0
Recall Score :0.9142857142857143
********************
Roc_Auc treino:1.0
Roc_Auc teste:0.9537909018355946
********************
Precisao Treino: 1.0
Precisao: Teste: 0.9411764705882353
********************


In [130]:
parametros = {
    "max_depth":[1,2,3,4,5,6,7,8,9,10],
    "min_samples_leaf":[5,10,15,20]
}

In [136]:
grid = GridSearchCV(estimator = tree ,param_grid = parametros,cv=5,scoring="accuracy")

In [137]:
grid.fit(x,y)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [5, 10, 15, 20]},
             scoring='accuracy')

In [140]:
result = pd.DataFrame(grid.cv_results_)

In [142]:
result.sort_values("rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
22,0.008484,0.000863,0.002095,0.000083,6,15,"{'max_depth': 6, 'min_samples_leaf': 15}",0.986,0.991,0.984,0.99,0.985,0.9872,0.002786,1
34,0.008382,0.000483,0.002117,0.000034,9,15,"{'max_depth': 9, 'min_samples_leaf': 15}",0.986,0.987,0.984,0.99,0.985,0.9864,0.002059,2
30,0.008085,0.000524,0.002063,0.000119,8,15,"{'max_depth': 8, 'min_samples_leaf': 15}",0.986,0.987,0.984,0.99,0.985,0.9864,0.002059,2
26,0.008082,0.000556,0.002012,0.000016,7,15,"{'max_depth': 7, 'min_samples_leaf': 15}",0.986,0.987,0.984,0.99,0.985,0.9864,0.002059,2
38,0.008311,0.000575,0.002099,0.000038,10,15,"{'max_depth': 10, 'min_samples_leaf': 15}",0.986,0.987,0.984,0.99,0.985,0.9864,0.002059,2


In [73]:
rf = RandomForestClassifier()

In [74]:
rf.fit(x_train,y_train)

RandomForestClassifier()

In [75]:
rf_pred = rf.predict(x_test)

In [76]:
x_test.assign(Valor_Real = y_test, Valor_Predito = rf_pred)

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities_Account,CD_Account,Online,CreditCard,Valor_Real,Valor_Predito
1501,1502,30,4,35,2,0.3,2,0,1,0,0,1,0,0
2586,2587,47,23,149,4,6.1,1,0,0,0,0,1,1,1
2653,2654,30,5,121,2,3.1,1,408,0,0,1,0,0,0
1055,1056,31,6,62,1,1.0,1,0,1,0,1,0,0,0
705,706,62,36,30,3,0.7,2,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,4712,65,40,59,3,2.4,1,0,0,0,0,0,0,0
2313,2314,58,32,54,3,0.3,2,0,0,0,1,1,0,0
3214,3215,61,37,33,3,0.1,3,0,0,0,1,0,0,0
2732,2733,33,9,38,1,2.1,3,133,0,0,0,0,0,0


In [109]:

print("METRICAS DO MODLEO RANDOM FOREST CLASSIFIER (PADRAO)")
print("*"*20)
print(f"Acurancia Train:{accuracy_score(y_train,rf.predict(x_train))}")
print(f"Acurancia teste:{accuracy_score(y_test,rf.predict(x_test))}")
print("*"*20)
print(f"Acuracia balancead Treino: {balanced_accuracy_score(y_train,rf.predict(x_train))}")
print(f"Acuracia balancead Teste: {balanced_accuracy_score(y_test,rf.predict(x_test))}")
print("*"*20)
print(f"Recall Score :{recall_score(y_train,rf.predict(x_train))}")
print(f"Recall Score :{recall_score(y_test,rf.predict(x_test))}")
print("*"*20)
print(f"Roc_Auc treino:{roc_auc_score(y_train,rf.predict(x_train))}")
print(f"Roc_Auc teste:{roc_auc_score(y_test,rf.predict(x_test))}")
print("*"*20)
print(f"Precisao Treino: {precision_score(y_train,rf.predict(x_train))}")
print(f"Precisao: Teste: {precision_score(y_test,rf.predict(x_test))}")
print("*"*20)

METRICAS DO MODLEO RANDOM FOREST CLASSIFIER (PADRAO)
********************
Acurancia Train:1.0
Acurancia teste:0.99
********************
Acuracia balancead Treino: 1.0
Acuracia balancead Teste: 0.9565841979249801
********************
Recall Score :1.0
Recall Score :0.9142857142857143
********************
Roc_Auc treino:1.0
Roc_Auc teste:0.9565841979249798
********************
Precisao Treino: 1.0
Precisao: Teste: 0.9896907216494846
********************




```
# Isto está formatado como código
```



In [171]:
parametros_rs ={
    "max_depth":[2,6,10]
}

In [172]:
rs = RandomizedSearchCV(rf,parametros_rs,n_iter=5,cv=5,scoring="accuracy")

In [173]:
rs.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 3 is smaller than n_iter=5. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'max_depth': [2, 6, 10]},
                   scoring='accuracy')

In [174]:
result_rs = pd.DataFrame(rs.cv_results_)

In [176]:
result_rs.sort_values("rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.391528,0.004350,0.011917,0.000288,10,{'max_depth': 10},0.987,0.984,0.985,0.989,0.985,0.9860,0.001789,1
1,0.336246,0.005800,0.011774,0.001382,6,{'max_depth': 6},0.985,0.980,0.981,0.985,0.976,0.9814,0.003382,2
0,0.342210,0.053707,0.011628,0.003209,2,{'max_depth': 2},0.917,0.907,0.914,0.916,0.905,0.9118,0.004874,3
